In [ ]:
# Boilerplate: This block goes into every notebook.
# It sets up the environment, installs the requirements, and checks for the required environment variables.

from IPython.display import clear_output
from dotenv import load_dotenv
import os

requirements_installed = False
max_retries = 3
retries = 0
REQUIRED_ENV_VARS = ["OPENAI_API_KEY", "TOOLHOUSE_API_KEY"]


def install_requirements():
    """Installs the requirements from requirements.txt file"""
    global requirements_installed
    if requirements_installed:
        print("Requirements already installed.")
        return

    print("Installing requirements...")
    install_status = os.system("pip install -r requirements.txt")
    if install_status == 0:
        print("Requirements installed successfully.")
        requirements_installed = True
    else:
        print("Failed to install requirements.")
        if retries < max_retries:
            print("Retrying...")
            retries += 1
            return install_requirements()
        exit(1)
    return


def setup_env():
    """Sets up the environment variables"""

    def check_env(env_var):
        value = os.getenv(env_var)
        if value is None:
            print(f"Please set the {env_var} environment variable.")
            exit(1)
        else:
            print(f"{env_var} is set.")

    load_dotenv(override=True)

    variables_to_check = REQUIRED_ENV_VARS

    for var in variables_to_check:
        check_env(var)


install_requirements()
clear_output()
setup_env()
print("🚀 Setup complete. Continue to the next cell.")

In [ ]:
## Example: Just a basic use of Toolhouse x OpenAI

import os
from openai import OpenAI
from toolhouse import Toolhouse

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TOOLHOUSE_API_KEY = os.getenv("TOOLHOUSE_API_KEY")

client = OpenAI(api_key=OPENAI_API_KEY)
th = Toolhouse(api_key=TOOLHOUSE_API_KEY, provider="openai")

MODEL = "gpt-4o-mini"

messages = [
    {
        "role": "user",
        "content": "Generate FizzBuzz code."
        "Execute it to show me the results up to 10.",
    }
]

response = client.chat.completions.create(
    model=MODEL, messages=messages, tools=th.get_tools()
)

messages += th.run_tools(response)

response = client.chat.completions.create(
    model=MODEL, messages=messages, tools=th.get_tools()
)

print(response.choices[0].message.content)

In [34]:
## Example: LLM with Toolhouse
## Include the following tools in the Toolhouse bundle: thp-system-design-assistant
## Sendgrid, Web Scraper, and Memory tools are included in this bundle.

import os
from openai import OpenAI
from toolhouse import Toolhouse
import traceback

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TOOLHOUSE_API_KEY = os.getenv("TOOLHOUSE_API_KEY")
MAX_ITERS = 10
th = Toolhouse(api_key=TOOLHOUSE_API_KEY, provider="openai")
openai = OpenAI(api_key=OPENAI_API_KEY)


def llm(
    prompt: str,
    model="gpt-4o-mini",
    system="You are a helpful AI assistant",
    intermediate_output=True,
    verbose=True,
) -> str:
    """Wrapper over Open AI LLM calls with Toolhouse Tools."""
    try:
        if verbose:
            print("System: ", system)
            print("Prompt: ", prompt)
        messages = [
            {"role": "system", "content": system},
            {"role": "user", "content": prompt},
        ]
        response = openai.chat.completions.create(
            model=model,
            messages=messages,
            tools=th.get_tools("thp-system-design-assistant"),
        )

        tool_calls = response.choices[0].message.tool_calls
        iters = 0
        content = response.choices[0].message.content

        while tool_calls and len(tool_calls) > 0 and iters < MAX_ITERS:
            if verbose or intermediate_output:
                if content:
                    print(f"Cycle {iters}: ", content)
                else:
                    print(f"Cycle {iters}: No content.")
            messages += th.run_tools(response)
            response = openai.chat.completions.create(
                model=model,
                messages=messages,
                tools=th.get_tools("thp-system-design-assistant"),
            )
            if verbose or intermediate_output:
                print(f"Generated intermediate response...")
            content = response.choices[0].message.content
            iters += 1
            tool_calls = response.choices[0].message.tool_calls

        if content and (verbose or intermediate_output):
            print(f"Final response: {content}")

        return content
    except Exception as e:
        traceback.print_exc()
        return "Sorry, failed to generate response.\nError: " + str(e)

In [ ]:
## Testing simple 'System Design' report emailer use case

topic = "Bloom Filters"
email = "contact.adityapatange@gmail.com"
model = "gpt-4o"

system = f"""
    You are THP System Design AI.
    An intelligent assistant that helps you understand system design concepts.
    You will provide detailed explanations of the topics you are asked to research.
    I will tell you the various things to research. 
    You will generate a detailed markdown report on the topic.
"""

prompt = f"""
    Provide a detailed explanation of {topic}.
    Include the following points:
    - What is {topic}?
    - How does '{topic}' work?
    - What are the use cases of '{topic}'?
    - Advantages and disadvantages of '{topic}'. (If applicable)
    - Industry examples of {topic}. 

    INSTRUCTIONS:
    - Generate a detailed markdown report on {topic}.
    This is for study purposes, so please provide a detailed explanation.
    - Generated a nicely formatted HTML email on your research with title 'The Hackers Playbook System Design: Case Studies on {topic}'.
    - Send the email to {email}
"""

response = llm(
    prompt, model=model, system=system, intermediate_output=True, verbose=True
)

print(response)